In [12]:
import os
import re
import glob
import numpy as np
import pandas as pd

from importlib import reload
from joebvp import cfg, utils, VPmeasure

In [13]:
def replace_string(dataset_name):
    segment = """lt_xspec [dataset_name]_x1d.fits 
lt_continuumfit --redshift 0.00053 [dataset_name]_x1d.fits [dataset_name]_continuumfit.fits
pyigm_igmguesses [dataset_name]_continuumfit.fits -o [dataset_name]_x1dfits_model.json
pyigm_fitdla --out_file [dataset_name]_xfitdla.json [dataset_name]_continuumfit.fits 0"""
    modified_segment = segment.replace("[dataset_name]", dataset_name)
    print(modified_segment)
    
replace_string("o6df24030")

lt_xspec o6df24030_x1d.fits 
lt_continuumfit --redshift 0.00053 o6df24030_x1d.fits o6df24030_continuumfit.fits
pyigm_igmguesses o6df24030_continuumfit.fits -o o6df24030_x1dfits_model.json
pyigm_fitdla --out_file o6df24030_xfitdla.json o6df24030_continuumfit.fits 0


In [14]:
reload(cfg)

d = pd.DataFrame({
    'instr': ['COS', 'COS', 'COS', 'COS', 'STIS', 'STIS', 'Gaussian'],
    'gratings': ['G130M', 'G160M', 'G185M', 'G225M', 'E230M', 'E140M', 'N/A'],
    'slits': ['NA', 'NA', 'NA', 'NA', '0.2x0.2', '0.2x0.2', 'NA'],
    'lsfranges': [[1100, 1460], [1400, 1800], [1800, 2100], [2100, 2278], [1607, 3129], [1144, 1729], [899, 1191]],
    'lps': ['1', '1', '1', '1', '1', '1', 'NA'],
    'cen_wave': ['1291', '1611', '1953', '2250', '1978', '1425', 'NA'],
    'pixel_scales': [None, None, None, None, None, None, 0.013], 
    'fwhms': [None, None, None, None, None, None, 0.02]
})

num = 5

cfg.lsfs = []
for col in ['instr', 'gratings', 'slits', 'cen_wave', 'pixel_scales', 'fwhms']:
    setattr(cfg, col, [d.at[num, col]])

cfg.lsfranges = np.array([d.at[num, 'lsfranges']])

In [15]:
base_directory = "/Users/billyli/Documents/float-for-morrow/SMC/NGC330-A02/E140M"
directories = sorted([d for d in os.listdir(base_directory) if d.startswith("E140M")])

In [16]:
for spectra_directory in directories:

    current_directory = os.path.join(base_directory, spectra_directory)

    os.chdir(current_directory)

    igm_model = glob.glob(os.path.join(current_directory, "*_x1dfits_model.json"))[0]
    continuumfit = glob.glob(os.path.join(current_directory, "*_continuumfit.fits"))[0]

    utils.pyigm_to_veeper(igm_model, continuumfit)

    os.chdir(base_directory)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [17]:
for spectra_directory in directories:

    current_directory = os.path.join(base_directory, spectra_directory)

    component_groups_dir = os.path.join(current_directory, "component_groups")
    output_dir = os.path.join(current_directory, "modified_component_groups")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    mc_lines_file = "/Users/billyli/Documents/float-for-morrow/mc_lines_e140m.txt"
    mc_lines_df = pd.read_csv(mc_lines_file, sep = r"\s+", header = None, names = ["line", "wavelength"])
    mc_lines_df["wavelength"] = mc_lines_df["wavelength"].astype(float)

    def is_valid_line(row, line_df):
        trans = row["trans"].strip()
        restwave = row["restwave"]
        matching_lines = line_df[line_df["line"] == trans]
        for wave in matching_lines["wavelength"]:
            if 0.997 * wave <= restwave <= 1.003 * wave:
                return True
        return False

    input_files = glob.glob(os.path.join(component_groups_dir, "*.txt"))

    for file in input_files:
        df = pd.read_csv(file, sep = "|")
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
        df["restwave"] = pd.to_numeric(df["restwave"], errors = "coerce")
        df["bval"] = pd.to_numeric(df["bval"], errors = "coerce")
    
        valid_mask = df.apply(lambda row: is_valid_line(row, mc_lines_df), axis = 1)
        df_filtered = df[valid_mask].copy()
    
        # df_filtered["col"] = df_filtered["col"] - 0.2
        df_filtered["bval"] = df_filtered["bval"]
    
        output_file = os.path.join(output_dir, os.path.basename(file))
        df_filtered.to_csv(output_file, sep = "|", index = False)
        # print(f"Processed file: {os.path.basename(file)} -> {output_file}")

In [18]:
# Before fitting, should manually adjust the input group files

In [19]:
for spectra_directory in directories:

    current_directory = os.path.join(base_directory, spectra_directory)

    os.chdir(current_directory)

    igm_models = glob.glob(os.path.join(current_directory, "*_x1dfits_model.json"))
    continuumfits = glob.glob(os.path.join(current_directory, "*_continuumfit.fits"))

    component_groups_dir = os.path.join(current_directory, "modified_component_groups")
    
    input_group_files = sorted([f for f in glob.glob(os.path.join(component_groups_dir, "input_group_*.txt")) if re.match(r".*input_group_\d+\.txt$", f)])

    VPmeasure.batch_fit(continuumfits[0], input_group_files, filepath = './modified_component_groups/')

    os.chdir(base_directory)

Bad METADATA;  proceeding without
`ftol` termination condition is satisfied.
Function evaluations 19, initial cost 1.6478e+02, final cost 1.5778e+02, first-order optimality 2.61e-02.

Fit results: 

1608.45	 0.000460	 15.513	 18.592	 -3.964
 	  	  	 0.091	 0.978	 1.134 

1611.2	 0.000460	 15.513	 18.592	 -3.964
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 3.253152
Iteration 1 -
`ftol` termination condition is satisfied.
Function evaluations 19, initial cost 1.6478e+02, final cost 1.5778e+02, first-order optimality 2.61e-02.

Fit results: 

1608.45	 0.000460	 15.513	 18.592	 -3.964
 	  	  	 0.091	 0.978	 1.134 

1611.2	 0.000460	 15.513	 18.592	 -3.964
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 3.253152
Iteration 2 -
Fit converged after 2 iterations.
VPmeasure: Fit converged: /Users/billyli/Documents/float-for-morrow/SMC/NGC330-A02/E140M/E140M10/modified_component_groups/input_group_0.txt
Line parameters written to:
/Users/billyli/Documents/float-for-morrow/SMC/NGC330-A02/E140M/E1

In [20]:
for spectra_directory in directories:

    current_directory = os.path.join(base_directory, spectra_directory)

    os.chdir(current_directory)

    df = pd.read_csv('compiledVPoutputs.dat', sep = '|')
    filtered_df = df[df['zsys'] > 0.0002]
    filtered_df = filtered_df[filtered_df['sigcol'] != 0.000]
    result = filtered_df[['col', 'sigcol', 'trans']]

    unique_result = result#.drop_duplicates()
    print(unique_result)
    os.chdir(base_directory)

      col  sigcol trans
0  15.933   0.052  S II
2  18.430   0.098   O I
3  14.217   0.029  NiII
6  15.513   0.091  FeII
8  16.219   0.059  MgII
       col  sigcol trans
0   16.871   0.842  S II
2   18.371   0.097   O I
3   14.304   0.037  NiII
6   15.730   0.074  FeII
8   16.348   0.068  MgII
10  12.809   0.342  CuII
       col  sigcol trans
0   16.217   0.185  S II
2   18.533   0.102   O I
3   14.227   0.034  NiII
6   15.670   0.087  FeII
8   16.441   0.098  MgII
10  12.848   0.392  CuII
       col  sigcol trans
0   16.728   0.519  S II
2   18.335   0.108   O I
3   14.231   0.025  NiII
6   15.981   0.058  FeII
8   16.165   0.093  MgII
10  12.867   0.139  CuII


In [21]:
directories = [d for d in os.listdir(base_directory)
               if os.path.isdir(os.path.join(base_directory, d))]

records = []

for specdir in directories:
    fn = os.path.join(base_directory, specdir, "compiledVPoutputs.dat")
    if not os.path.isfile(fn):
        continue

    df = pd.read_csv(fn, sep = "|")
    df = df[(df["zsys"] > 0.0002) & (df["sigcol"] != 0.0)]
    records.append(df[["trans", "col", "sigcol"]])

all_measurements = pd.concat(records, ignore_index = True)

def random_effects_mean(vals, sigmas):
    vals = np.asarray(vals, dtype = float)
    sigmas = np.asarray(sigmas, dtype = float)
    var = sigmas ** 2

    w  = 1.0 / var
    mu = (w * vals).sum() / w.sum()

    Q  = (w * (vals - mu) ** 2).sum()
    df = len(vals) - 1
    c  = w.sum() - (w ** 2).sum() / w.sum()
    T2 = max(0.0, (Q - df) / c) if c > 0 else 0.0

    w_re = 1.0 / (var + T2)
    mean = (w_re * vals).sum() / w_re.sum()
    std  = np.sqrt(1.0 / w_re.sum())
    return mean, std

combined = (
    all_measurements
    .groupby("trans", sort = True)
    .apply(lambda g: random_effects_mean(g["col"], g["sigcol"]))
    .apply(pd.Series)
    .reset_index()
    .rename(columns = {0: "col_combined", 1: "sigcol_combined"})
)

combined = combined.sort_values("trans").reset_index(drop = True)

for _, row in combined.iterrows():
    print(f"{row.trans}  {row.col_combined:.3f} ± {row.sigcol_combined:.3f}")

CuII  12.858 ± 0.122
FeII  15.731 ± 0.104
MgII  16.288 ± 0.056
NiII  14.240 ± 0.017
O I  18.418 ± 0.050
S II  16.118 ± 0.155


/var/folders/7n/v6gcxcpj68q6nnv2znnc85xc0000gn/T/ipykernel_1021/2562963298.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: random_effects_mean(g["col"], g["sigcol"]))


In [22]:
# END